In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))
from wasserstein import Spectrum, NMRSpectrum

from typing import List
from matplotlib import pyplot as plt

import numpy as np
from tqdm import trange

components_names = ["Pinene", "Benzyl benzoate"]

protons_list = [16, 12]

filename = "preprocessed_mix.csv"
mix = np.loadtxt(filename, delimiter=",")
# If you are using file exported from Mnova, comment line above and uncomment line below.
# mix = np.loadtxt(filename, delimiter='\t', usecols=[0,1])

how_many_components = len(components_names)
names = ["comp" + str(i) for i in range(how_many_components)]

files_with_components = ["preprocessed_comp0.csv", "preprocessed_comp1.csv"]
spectra = []
for i in range(how_many_components):
    filename = files_with_components[i]
    spectra.append(np.loadtxt(filename, delimiter=","))
    # If you are using file exported from Mnova, comment line above and uncomment line below.
    # spectra.append(np.loadtxt(filename, delimiter='\t', usecols=[0,1]))

spectra2: List[NMRSpectrum] = []
names = []
for i in range(len(spectra)):
    spectra2.append(
        NMRSpectrum(
            confs=list(zip(spectra[i][:, 0], spectra[i][:, 1])), protons=protons_list[i]
        )
    )
    names.append("comp" + str(i))

spectra = spectra2
del spectra2
mix = NMRSpectrum(confs=list(zip(mix[:, 0], mix[:, 1])))
mix.trim_negative_intensities()
mix.normalize()
for sp in spectra:
    sp.trim_negative_intensities()
    sp.normalize()
plt.title("Mixture")
mix.plot(profile=True)
for i, sp in enumerate(spectra):
    plt.title("Component " + str(i))
    sp.plot(profile=True)
print(spectra[0].confs[:10])


In [3]:
import numpy as np

linsapce = np.linspace(1, 301, 9)
print(linsapce)

[  1.   38.5  76.  113.5 151.  188.5 226.  263.5 301. ]


In [ ]:
def filter_significant_features(first, second, n_features):
    """
    Select the n most significant features from both spectra.
    Uses combined intensity to determine importance.
    """
    # Create dictionary with combined intensities
    combined_intensities = {}
    for mz, prob in first.confs:
        combined_intensities[mz] = combined_intensities.get(mz, 0) + prob
    for mz, prob in second.confs:
        combined_intensities[mz] = combined_intensities.get(mz, 0) + prob

    # Select top n features
    selected_features = set(
        sorted(
            combined_intensities.keys(),
            key=lambda mz: combined_intensities[mz],
            reverse=True,
        )[:n_features]
    )

    # Filter spectra
    filtered_first = first.copy()
    filtered_first.confs = [
        (mz, prob) for mz, prob in first.confs if mz in selected_features
    ]

    filtered_second = second.copy()
    filtered_second.confs = [
        (mz, prob) for mz, prob in second.confs if mz in selected_features
    ]

    # Re-normalize
    filtered_first.normalize()
    filtered_second.normalize()

    return filtered_first, filtered_second


# Use only n most significant features
spectra_damped = [None, None]
spectra_damped[0], spectra_damped[1] = filter_significant_features(spectra[0], spectra[1], 2000)
distance = spectra_damped[0].WSDistance(spectra_damped[1])
# distance = WSDistance(spectra[0], spectra[1], n_features=2000)
difference = spectra[0].WSDistance(spectra[1]) - distance
print(distance)
print(difference)

gradient = spectra_damped[0].WSGradient(spectra_damped[1])
print(gradient)

In [ ]:
import matplotlib.pyplot as plt
from tqdm import trange

def calculate_gradient(mix, comp0, comp1, p, epsilon=1e-5):
    delta = epsilon

    p_plus = np.array([p[0] + delta, p[1] - delta])
    p_minus = np.array([p[0] - delta, p[1] + delta])

    p_plus = np.clip(p_plus, 1e-6, 1)
    p_plus /= p_plus.sum()

    p_minus = np.clip(p_minus, 1e-6, 1)
    p_minus /= p_minus.sum()

    sp_plus = Spectrum.ScalarProduct([comp0, comp1], p_plus)
    sp_minus = Spectrum.ScalarProduct([comp0, comp1], p_minus)

    sp_plus.normalize()
    sp_minus.normalize()

    grad = (mix.WSDistance(sp_plus) - mix.WSDistance(sp_minus)) / (2 * epsilon)

    # Gradient of p[1] is -grad because p[1] = 1 - p[0]
    grad_vec = np.array([grad, -grad])

    return grad_vec


def mirror_descent_two_weights(
    mix, comp0, comp1, learning_rate=1.0, T=100, epsilon=1e-5
):
    p = np.array([0.5, 0.5])  # start from uniform mixture
    history = [p.copy()]
    scores = []

    for _ in trange(T, desc="Mirror Descent (2 weights)"):
        # Track score (distance from current estimate to true mixture)
        estimated_mix = Spectrum.ScalarProduct([comp0, comp1], p)
        estimated_mix.normalize()
        ws = mix.WSDistance(estimated_mix)
        scores.append(ws)

        # Compute gradient
        grad_vec = calculate_gradient(mix, comp0, comp1, p, epsilon)

        # Mirror descent update
        w = p * np.exp(-learning_rate * grad_vec)
        p = w / w.sum()

        history.append(p.copy())

    return p, np.array(history), np.array(scores)


# Run mirror descent
# mix.normalize()
# spectra[0].normalize()
# spectra[1].normalize()

final_p, traj, score_history = mirror_descent_two_weights(
    mix, spectra[0], spectra[1], learning_rate=0.0002, T=100
)

# Reconstruct mixture from estimated weights
estimated_mix = Spectrum.ScalarProduct([spectra[0], spectra[1]], final_p)
estimated_mix.normalize()

# Compute Wasserstein distance to the actual mixture
ws_dist = mix.WSDistance(estimated_mix)

# Compute Wasserstein distance for the given mixture
given_p = np.array([0.393072, 0.606928])
given_mix = Spectrum.ScalarProduct([spectra[0], spectra[1]], given_p)
given_mix.normalize()
ws_dist_given = mix.WSDistance(given_mix)

# Print results
print("\nFinal weights:")
print(f"  comp1 = {final_p[0]:.4f}")
print(f"  comp2 = {final_p[1]:.4f}")
print(f"\nFinal Wasserstein distance to true mixture: {ws_dist:.6f}")
print(
    f"\nWasserstein distance for given mixture (0.393072, 0.606928): {ws_dist_given:.6f}"
)

plt.plot(traj[:, 0], label="Component 0")
plt.plot(traj[:, 1], label="Component 1")
plt.xlabel("Iteration")
plt.ylabel("Weight")
plt.title("Mirror Descent Weight Evolution")
plt.legend()
plt.grid(True)
plt.show()

plt.plot(score_history, marker="o")
plt.xlabel("Iteration")
plt.ylabel("Wasserstein Distance")
plt.title("Distance to True Mixture Over Iterations")
plt.grid(True)
plt.show()


In [ ]:
import torch

def numpy_to_torch_tensor(pairs):
    return {
        "values": torch.tensor([float(v) for v, _ in pairs], dtype=torch.float64, requires_grad=True),
        "probs": torch.tensor([float(p) for _, p in pairs], dtype=torch.float64, requires_grad=True)
    }

spectra_torch = [numpy_to_torch_tensor(sp.confs) for sp in spectra]
mix_torch = numpy_to_torch_tensor(mix.confs)

print(spectra_torch[0]["probs"][:10])
print(spectra_torch[0]["values"][:10])


In [ ]:
def weighted_quantile_torch(spectre, quantiles):
    # assume data normalized
    values, probs = spectre["values"], spectre["probs"]
    
    sorted_indices = torch.argsort(values)
    values = values[sorted_indices]
    probs = probs[sorted_indices]
    
    cum_probs = torch.cumsum(probs, dim=0)

    indices = torch.searchsorted(cum_probs, quantiles, right=True)
    indices = torch.clamp(indices, 0, len(values) - 1)
    quantile_values = values[indices]
    
    return quantile_values

data = spectra_torch[0]
quantiles = [0.25, 0.5, 0.75]
quantiles = torch.tensor(quantiles, dtype=torch.float64)
result = weighted_quantile_torch(data, quantiles)
print(result)

In [ ]:
def wasserstein_distance(mu, nu, p=1):
    _, mu_probs = mu["values"], mu["probs"]
    _, nu_probs = nu["values"], nu["probs"]

    cum_probs_mu = torch.cumsum(mu_probs, dim=0)
    cum_probs_nu = torch.cumsum(nu_probs, dim=0)
    
    t = torch.cat([cum_probs_mu, cum_probs_nu])
    t, _ = torch.sort(t)
    
    F_mu_inv = weighted_quantile_torch(mu, t)
    F_nu_inv = weighted_quantile_torch(nu, t)
    
    integral = torch.trapz(torch.abs(F_mu_inv - F_nu_inv) ** p, t)
    return integral ** (1 / p)


ws = wasserstein_distance(spectra_torch[0], spectra_torch[1])
print(ws)

In [ ]:
def ws_two_mix(p):
    mu_values, mu_probs = spectra_torch[0]["values"], spectra_torch[0]["probs"]
    nu_values, nu_probs = spectra_torch[1]["values"], spectra_torch[1]["probs"]
    
    estimated_values = mu_values * p[0] + nu_values * p[1]
    estimated_probs = mu_probs * p[0] + nu_probs * p[1]
    estimated_probs /= estimated_probs.sum()
    
    estimated_mix = {
        "values": estimated_values,
        "probs": estimated_probs
    }
    ws = wasserstein_distance(mix_torch, estimated_mix)
    return ws

p = torch.tensor([0.5, 0.5], dtype=torch.float64, requires_grad=True)

ws = ws_two_mix(p)
ws.backward()
print(ws)
print(p.grad)

In [ ]:
def mirror_descent_torch(learning_rate=1.0, T=100):
    p = torch.tensor([0.5, 0.5], dtype=torch.float64, requires_grad=True)
    history = [p.clone().detach()]
    scores = []

    for _ in trange(T, desc="Mirror Descent (2 weights)"):
        ws = ws_two_mix(p)
        ws.backward()
        scores.append(ws.item())

        # Mirror descent update
        with torch.no_grad():
            w = p * torch.exp(-learning_rate * p.grad)
            p.copy_(w / w.sum())

        p.grad.zero_()  # Reset gradients
        history.append(p.clone().detach())

    return p, history, scores

final_p, traj, score_history = mirror_descent_torch(learning_rate=0.001, T=100)

# plot scores
plt.plot(score_history, marker="o")
plt.xlabel("Iteration")
plt.ylabel("Wasserstein Distance")
plt.title("Distance to True Mixture Over Iterations")
plt.grid(True)
plt.show()

In [ ]:
import ot

N = 800

def signif_features(spectrum, n_features):
    spectrum_confs = sorted(spectrum.confs, key=lambda x: x[1], reverse=True)[:n_features]
    spectrum_signif = NMRSpectrum(confs=spectrum_confs, protons=spectrum.protons)
    spectrum_signif.normalize()
    return spectrum_signif

spectra_signif = [None] * 2
spectra_signif[0] = signif_features(spectra[0], N)
spectra_signif[1] = signif_features(spectra[1], N)
mix_signif = signif_features(mix, 2*N)

for i, sp in enumerate(spectra):
    plt.title("Component " + str(i))
    sp.plot(profile=True)
    plt.title("Component " + str(i) + " signif")
    spectra_signif[i].plot(profile=True)

plt.title("Mixture")
mix.plot(profile=True)
plt.title("Mixture signif")
mix_signif.plot(profile=True)

given_p = np.array([0.5, 0.5])
given_mix = Spectrum.ScalarProduct([spectra_signif[0], spectra_signif[1]], given_p)
given_mix.normalize()
plt.title("Mixed signif 50-50")
given_mix.plot()

given_mix_full = Spectrum.ScalarProduct([spectra[0], spectra[1]], given_p)
given_mix_full.normalize()


In [ ]:
def cost_matrix(spectrum1, spectrum2):
    vals1 = [val for val, _ in spectrum1.confs]
    vals2 = [val for val, _ in spectrum2.confs]
    
    cost = np.zeros((len(vals1), len(vals2)))
    # cost = np.full((len(vals1), len(vals2)), np.inf)

    for i, v1 in enumerate(vals1):
        for j, v2 in enumerate(vals2):
            cost[i, j] = v1 - v2
    
    return cost

# print(cost_matrix(spectra_signif[0], spectra_signif[1]))

In [ ]:
from scipy.sparse import coo_matrix

def sparse_cost_matrix(spectrum1, spectrum2, threshold=None, block_size=1000):
    vals1 = np.array([val for val, _ in spectrum1.confs])
    vals2 = np.array([val for val, _ in spectrum2.confs])

    n1, n2 = len(vals1), len(vals2)
    data, rows, cols = [], [], []

    for i_start in trange(0, n1, block_size):
        i_end = min(i_start + block_size, n1)
        v1_block = vals1[i_start:i_end]

        for j_start in range(0, n2, block_size):
            j_end = min(j_start + block_size, n2)
            v2_block = vals2[j_start:j_end]

            # Broadcasted block subtraction
            block_diff = v1_block[:, None] - v2_block[None, :]

            if threshold is not None:
                mask = np.abs(block_diff) <= threshold
            else:
                mask = np.ones_like(block_diff, dtype=bool)

            bi, bj = np.nonzero(mask)
            data.append(block_diff[bi, bj])
            rows.append(bi + i_start)
            cols.append(bj + j_start)

    # Concatenate all data
    data = np.concatenate(data)
    rows = np.concatenate(rows)
    cols = np.concatenate(cols)

    sparse_cost = coo_matrix((data, (rows, cols)), shape=(n1, n2)).tocsr()
    return sparse_cost

In [ ]:

a = np.array([p for _, p in mix_signif.confs])
b = np.array([p for _, p in given_mix.confs])
M = cost_matrix(mix_signif, given_mix)

G0, log = ot.unbalanced.lbfgsb_unbalanced(a, b, M, reg=0, reg_m=5, reg_div='kl', regm_div='tv', log=True)
print(log["res"], log["cost"], log["total_cost"])
# print(ot.unbalanced.lbfgsb_unbalanced2(a, b, M, reg=5, reg_m=5, reg_div='kl', regm_div='kl'))
# print(ot.unbalanced.lbfgsb_unbalanced2(a, b, M, reg=5, reg_m=5, reg_div='l2', regm_div='tv'))
# print(ot.unbalanced.lbfgsb_unbalanced2(a, b, M, reg=5, reg_m=5, reg_div='kl', regm_div='tv'))
print(mix_signif.WSDistance(given_mix))


# a2 = np.array([p for _, p in mix.confs])
# b2 = np.array([p for _, p in given_mix_full.confs])
# M_sparse = sparse_cost_matrix(mix, given_mix_full, threshold=0.01)
# M2 = M_sparse.toarray()
# print(M2.shape)
# print(ot.unbalanced.lbfgsb_unbalanced2(a2, b2, M2, reg=0, reg_m=5, reg_div='kl', regm_div='tv'))

In [ ]:
import matplotlib.pyplot as plt
from tqdm import trange

def calculate_gradient_ot(mix, comp0, comp1, p, G0=None, epsilon=1e-5):
    delta = epsilon

    p_plus = np.array([p[0] + delta, p[1] - delta])
    p_minus = np.array([p[0] - delta, p[1] + delta])

    p_plus = np.clip(p_plus, 1e-6, 1)
    p_plus /= p_plus.sum()

    p_minus = np.clip(p_minus, 1e-6, 1)
    p_minus /= p_minus.sum()

    sp_plus = Spectrum.ScalarProduct([comp0, comp1], p_plus)
    sp_minus = Spectrum.ScalarProduct([comp0, comp1], p_minus)

    sp_plus.normalize()
    sp_minus.normalize()

    # grad = (mix.WSDistance(sp_plus) - mix.WSDistance(sp_minus)) / (2 * epsilon)

    a = np.array([p for _, p in mix.confs])
    b1 = np.array([p for _, p in sp_plus.confs])
    b2 = np.array([p for _, p in sp_minus.confs])
    M1 = cost_matrix(mix, sp_plus)
    M2 = cost_matrix(mix, sp_minus)

    ws_plus = ot.unbalanced.lbfgsb_unbalanced2(a, b1, M1, reg=0, reg_m=5, reg_div='kl', regm_div='tv')
    ws_minus = ot.unbalanced.lbfgsb_unbalanced2(a, b2, M2, reg=0, reg_m=5, reg_div='kl', regm_div='tv')

    grad = (ws_plus - ws_minus) / (2 * epsilon)
    # Gradient of p[1] is -grad because p[1] = 1 - p[0]
    grad_vec = np.array([grad, -grad])

    return grad_vec


def mirror_descent_ot(
    mix, comp0, comp1, learning_rate=1.0, T=100, epsilon=1e-5, gamma=0.98
):
    p = np.array([0.5, 0.5])
    history = [p.copy()]
    scores = []
    G0 = None

    for _ in trange(T, desc="Mirror Descent (2 weights)"):
        estimated_mix = Spectrum.ScalarProduct([comp0, comp1], p)
        estimated_mix.normalize()

        a = np.array([p for _, p in mix.confs])
        b = np.array([p for _, p in estimated_mix.confs])

        M = cost_matrix(mix, estimated_mix)

        G0, log = ot.unbalanced.lbfgsb_unbalanced(a, b, M, reg=0, reg_m=5, reg_div='kl', regm_div='tv', log=True)
        ws = log["cost"]
        print(log["res"]["nit"])
        print(log["res"]["fun"])
        print(ws)
        # ws = abs(ot.unbalanced.lbfgsb_unbalanced2(a, b, M, reg=0, reg_m=5, reg_div='kl', regm_div='tv'))
        scores.append(ws)

        grad_vec = calculate_gradient_ot(mix, comp0, comp1, p, epsilon)

        w = p * np.exp(-learning_rate * grad_vec)
        p = w / w.sum()

        learning_rate *= gamma

        history.append(p.copy())

    return p, np.array(history), np.array(scores)

final_p, traj, score_history = mirror_descent_ot(
    mix_signif, spectra_signif[0], spectra_signif[1], learning_rate=0.02, T=40, gamma=0.98
)

print("\nFinal weights:")
print(f"  comp1 = {final_p[0]:.4f}")
print(f"  comp2 = {final_p[1]:.4f}")

plt.plot(traj[:, 0], label="Component 0")
plt.plot(traj[:, 1], label="Component 1")
plt.xlabel("Iteration")
plt.ylabel("Weight")
plt.title("Mirror Descent Weight Evolution")
plt.legend()
plt.grid(True)
plt.show()

plt.plot(score_history, marker="o")
plt.xlabel("Iteration")
plt.ylabel("Wasserstein Distance")
plt.title("Distance to True Mixture Over Iterations")
plt.grid(True)
plt.show()

In [ ]:
a = np.array([p for _, p in mix_signif.confs])
b = np.array([p for _, p in given_mix.confs])
M = cost_matrix(mix_signif, given_mix)

G0 = a[:, None] * b[None, :]

ws1 = np.sum(G0 * M)
ws2 = (mix_signif.WSDistance(given_mix))

print(ws1, ws2)

def ws_naive(spectrum1, spectrum2, block_size=700):
    vals1 = np.array([val for val, _ in spectrum1.confs])
    a = np.array([p for _, p in spectrum1.confs])

    vals2 = np.array([val for val, _ in spectrum2.confs])
    b = np.array([p for _, p in spectrum2.confs])

    ws = 0.0

    for i_start in trange(0, len(vals1), block_size):
        i_end = min(i_start + block_size, len(vals1))
        v1_block = vals1[i_start:i_end]
        a_block = a[i_start:i_end]

        # Broadcasting cost difference and G0 incrementally
        diff_block = v1_block[:, None] - vals2[None, :]
        prob_block = a_block[:, None] * b[None, :]

        ws += np.sum(prob_block * diff_block)

    return ws

def ws_direct(spectrum1, spectrum2):
    vals1 = np.array([val for val, _ in spectrum1.confs])
    a = np.array([p for _, p in spectrum1.confs])

    vals2 = np.array([val for val, _ in spectrum2.confs])
    b = np.array([p for _, p in spectrum2.confs])

    # Normalize in case the inputs are not exact probabilities
    a /= a.sum()
    b /= b.sum()

    return abs(np.dot(a, vals1) - np.dot(b, vals2))



# ws1_full = ws_naive(mix, given_mix_full)
ws2_full = mix.WSDistance(given_mix_full)
ws1_full = ws_direct(mix, given_mix_full)

print(len(mix.confs), len(given_mix_full.confs))
print(ws1_full, ws2_full)


# f(p) = ws_direct(mix, p*s1 + (1-p)*s2)

In [ ]:
def calculate_gradient_naive(mix, comp0, comp1, p, epsilon=1e-5):
    grad_vec = np.zeros_like(p)

    for i in range(len(p)):
        # Create perturbed copies
        p_plus = p.copy()
        p_minus = p.copy()

        p_plus[i] += epsilon
        p_minus[i] -= epsilon

        # Clip and renormalize
        p_plus = np.clip(p_plus, 1e-6, 1)
        p_plus /= p_plus.sum()

        p_minus = np.clip(p_minus, 1e-6, 1)
        p_minus /= p_minus.sum()

        # Create mixed spectra
        sp_plus = Spectrum.ScalarProduct([comp0, comp1], p_plus)
        sp_minus = Spectrum.ScalarProduct([comp0, comp1], p_minus)

        sp_plus.normalize()
        sp_minus.normalize()

        # Finite difference for the i-th coordinate
        grad_vec[i] = (ws_direct(mix, sp_plus) - ws_direct(mix, sp_minus)) / (2 * epsilon)

    return grad_vec


def mirror_descent_naive(
    mix, comp0, comp1, learning_rate=1.0, T=100, epsilon=1e-5, gamma=0.8
):
    p = np.array([0.5, 0.5])
    history = [p.copy()]
    scores = []

    for _ in trange(T, desc="Mirror Descent (2 weights)"):
        estimated_mix = Spectrum.ScalarProduct([comp0, comp1], p)
        estimated_mix.normalize()

        scores.append(ws_direct(mix, estimated_mix))
        grad_vec = calculate_gradient_naive(mix, comp0, comp1, p, epsilon)

        w = p * np.exp(-learning_rate * grad_vec)
        p = w / w.sum()

        learning_rate *= gamma

        history.append(p.copy())

    return p, np.array(history), np.array(scores)

final_p, traj, score_history = mirror_descent_naive(
    mix, spectra[0], spectra[1], learning_rate=0.005, T=50, gamma=0.90
)

print("\nFinal weights:")
print(f"  comp1 = {final_p[0]:.4f}")
print(f"  comp2 = {final_p[1]:.4f}")

plt.plot(traj[:, 0], label="Component 0")
plt.plot(traj[:, 1], label="Component 1")
plt.xlabel("Iteration")
plt.ylabel("Weight")
plt.title("Mirror Descent Weight Evolution")
plt.legend()
plt.grid(True)
plt.show()

plt.plot(score_history, marker="o")
plt.xlabel("Iteration")
plt.ylabel("Wasserstein Distance")
plt.title("Distance to True Mixture Over Iterations")
plt.grid(True)
plt.show()

In [ ]:
def true_cost_matrix(spectrum1, spectrum2, threshhold=5.0):
    vals1 = [val for val, _ in spectrum1.confs]
    vals2 = [val for val, _ in spectrum2.confs]
    
    # cost = np.full((len(vals1), len(vals2)), 0)
    cost = np.zeros((len(vals1), len(vals2)))
    
    for i, v1 in enumerate(vals1):
        for j, v2 in enumerate(vals2):
            # if abs(v1 - v2) <= threshhold:
            cost[i, j] = v1 - v2
    
    return cost

TEST_N = 200

# 0.011216905452141613
# 0.010568778251053077

true_p = np.array([0.5, 0.5])
aprox_mix = Spectrum.ScalarProduct([signif_features(spectra[0], TEST_N), signif_features(spectra[1], TEST_N)], true_p)
aprox_mix.normalize()
mix_test = signif_features(mix, 2*TEST_N)

p1 = np.array([p for _, p in mix_test.confs])
v1 = np.array([v for v, _ in mix_test.confs])
p2 = np.array([p for _, p in aprox_mix.confs])
v2 = np.array([v for v, _ in aprox_mix.confs])
M_test = true_cost_matrix(mix_test, aprox_mix)
reg_m = np.array([0, 0])

given_mix_full = Spectrum.ScalarProduct([spectra[0], spectra[1]], true_p)
given_mix_full.normalize()

In [ ]:
G0, log = ot.unbalanced.lbfgsb_unbalanced(p1, p2, M_test, reg=0, reg_m=reg_m, reg_div='kl', regm_div='tv', log=True, verbose=True, numItermax=1500)
print(log["res"])
print(log["cost"], log["total_cost"])

In [ ]:
print(mix_test.WSDistance(aprox_mix))
print(mix.WSDistance(given_mix_full))

In [ ]:
from scipy.stats import wasserstein_distance

print(wasserstein_distance(v1, v2, p1, p2))

p1_full = np.array([p for _, p in mix.confs])
v1_full = np.array([v for v, _ in mix.confs])
p2_full = np.array([p for _, p in given_mix_full.confs])
v2_full = np.array([v for v, _ in given_mix_full.confs])

print(wasserstein_distance(p1_full, p2_full, v1_full, v2_full))

In [ ]:
G0 = p1[:, None] * p2[None, :]

print(np.sum(G0 * M_test)) # E_p1(v1) - E_p2(v2)

In [ ]:
print(ot.emd2(p1, p2, M_test))